In [3]:
from simulator.game.connect import Config  # type: ignore[import]

from alphazero_implementation.models.games.connect4 import CNNModel

%load_ext autoreload
%autoreload 2


# from simulator.textual.examples.agent import AgentApp, RandomAgent

config = Config(6, 7, 4)

path = "/Users/pveron/Code/alphazero-implementation/checkpoints/run_150/model-epoch=2199.ckpt"
path = "/Users/pveron/Code/alphazero-implementation/checkpoints/run_156/model-epoch=1999.ckpt"
path = "/Users/pveron/Code/alphazero-implementation/lightning_logs/alphazero/run_168_iter200_episodes100_sims100/checkpoints/epoch=1999-step=1431360.ckpt"

model = CNNModel.load_from_checkpoint(  # type: ignore[arg-type]
    path,
    height=config.height,
    width=config.width,
    max_actions=config.width,
    num_players=config.num_players,
)
# model = CNNModel(  # type: ignore[arg-type]
#     height=config.height,
#     width=config.width,
#     max_actions=config.width,
#     num_players=config.num_players,
# )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
state = config.sample_initial_state()
[policy], [value] = model.predict([state])

print(policy.values())
print(value)


dict_values([0.38212865591049194, 0.018891004845499992, 0.03477592393755913, 0.008441480807960033, 0.004113901872187853, 0.3447142243385315, 0.20693479478359222])
[0.07656644284725189, -0.0765664279460907]


In [5]:
state.to_json()

{'config': {'count': 4, 'height': 6, 'width': 7},
 'grid': [[-1, -1, -1, -1, -1, -1, -1],
  [-1, -1, -1, -1, -1, -1, -1],
  [-1, -1, -1, -1, -1, -1, -1],
  [-1, -1, -1, -1, -1, -1, -1],
  [-1, -1, -1, -1, -1, -1, -1],
  [-1, -1, -1, -1, -1, -1, -1]],
 'player': 0}

In [34]:
json = {
    "config": {"count": 4, "height": 6, "width": 7},
    "grid": [
        [0, 0, -1, 1, -1, 0, -1],
        [-1, -1, -1, 1, -1, -1, -1],
        [-1, -1, -1, 1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
    ],
    "player": 0,
}

In [35]:
from simulator.game.connect import State

state = State.from_json(json)

In [61]:
[policy], [value] = model.predict([state])

print(policy.values())
print(value)

dict_values([0.010334096848964691, 0.0022563186939805746, 0.0013825338101014495, 0.006581863854080439, 0.00029155719676055014, 0.9764062762260437, 0.0027472558431327343])
[-0.5397583246231079, 0.5397583842277527]


In [37]:
from alphazero_implementation.mcts.agent import MCTSAgent

agent = MCTSAgent(model, 2, 100, config.sample_initial_state())

model.eval()

[episode_1, episode_2] = agent.generate_episodes()

# print(episode)
# print(episode.samples[4])
#     pass
#     print(episode)
#     print(episode.samples[4])


In [38]:
episode_1.samples[-1].state.grid, episode_1.samples[-1].state.has_ended


(array([[ 1,  1,  1,  1,  0,  0, -1],
        [-1, -1,  0, -1, -1,  0, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1]], dtype=int8),
 True)

In [39]:
episode_2.samples[-1].state.grid, episode_2.samples[-1].state.has_ended


(array([[ 1,  1,  1,  1,  0,  0, -1],
        [-1, -1, -1,  0, -1,  0, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1]], dtype=int8),
 True)

In [40]:
for action in episode_2.samples[-1].state.actions:
    print(action.column)


In [41]:
episode_2.samples[-1].state.has_ended

True

In [59]:
from alphazero_implementation.mcts.mcgs import Node

path = "/Users/pveron/Code/alphazero-implementation/checkpoints/run_156/model-epoch=1999.ckpt"
path = "/Users/pveron/Code/alphazero-implementation/lightning_logs/alphazero/run_168_iter200_episodes100_sims100/checkpoints/epoch=1999-step=1431360.ckpt"


model = CNNModel.load_from_checkpoint(  # type: ignore[arg-type]
    path,
    height=config.height,
    width=config.width,
    max_actions=config.width,
    num_players=config.num_players,
)

model.eval()

json = {
    "config": {"count": 4, "height": 6, "width": 7},
    "grid": [
        [0, 0, 0, -1, -1, 1, -1],
        [-1, -1, -1, 1, -1, 1, -1],
        [-1, -1, -1, 1, -1, 1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
    ],
    "player": 0,
}

state = State.from_json(json)


In [60]:
policy = model.predict([state])[0][0]
action = max(policy.items(), key=lambda x: x[1])[0]
action.column

5

In [44]:
action.sample_next_state().grid


array([[ 0,  0,  0, -1, -1,  1, -1],
       [ 0, -1, -1,  1, -1,  1, -1],
       [-1, -1, -1,  1, -1,  1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1]], dtype=int8)

In [45]:
agent = MCTSAgent(
    model,
    num_episodes=100,
    simulations_per_episode=100,
    initial_state=state,
)
policy = agent.compute_policy(Node(state), {})
action = max(policy.items(), key=lambda x: x[1])[0]
action.column

0

In [46]:
action.sample_next_state().grid

array([[ 0,  0,  0, -1, -1,  1, -1],
       [ 0, -1, -1,  1, -1,  1, -1],
       [-1, -1, -1,  1, -1,  1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1]], dtype=int8)

In [47]:
json = {
    "config": {"count": 4, "height": 6, "width": 7},
    "grid": [
        [0, 0, 0, -1, -1, 1, -1],
        [-1, -1, -1, 1, -1, 1, -1],
        [-1, -1, -1, 1, -1, 1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
    ],
    "player": 0,
}

state = State.from_json(json)
state.grid


array([[ 0,  0,  0, -1, -1,  1, -1],
       [-1, -1, -1,  1, -1,  1, -1],
       [-1, -1, -1,  1, -1,  1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1]], dtype=int8)

In [48]:
state.action_at(3).sample_next_state().reward

array([ 1., -1.], dtype=float32)

In [49]:
from simulator.game.connect import Action

from alphazero_implementation.mcts.mcgs import Node

json = {
    "config": {"count": 4, "height": 6, "width": 7},
    "grid": [
        [0, 0, 0, -1, -1, 1, -1],
        [-1, -1, -1, 1, -1, 1, -1],
        [-1, -1, -1, 1, -1, 1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
    ],
    "player": 0,
}

state = State.from_json(json)

current_node = Node(state)

for _ in range(100):
    node: Node = current_node
    path: list[tuple[Node, Action]] = []
    nodes_by_state = {}

    while True:
        if node.is_terminal:
            node.utility_values = node.game_state.reward.tolist()  # type: ignore[attr-defined]
            # Add terminal nodes to leaf_nodes for backpropagation
            break
        elif node.visit_count == 0:  # New node not yet visited
            [policy], [value] = model.predict([node.game_state])
            node.action_policy = policy
            node.utility_values = value
            break
        else:
            action = node.select_action()
            if action not in node.children_and_edge_visits:
                new_game_state = action.sample_next_state()

                if new_game_state in nodes_by_state:
                    child = nodes_by_state[new_game_state]
                else:
                    child = Node(game_state=new_game_state)
                    nodes_by_state[new_game_state] = child

                node.children_and_edge_visits[action] = (child, 0)
            else:
                child = node.children_and_edge_visits[action][0]
            path.append((node, action))
            node = child

    node.visit_count = 1
    node.cumulative_value = node.utility_values[node.game_state.player]

    # Backpropagate all paths
    for parent, action in reversed(path):
        # Update edge visits
        child, edge_visits = parent.children_and_edge_visits[action]
        parent.children_and_edge_visits[action] = (child, edge_visits + 1)

        # Update visit count and cumulative value
        children_and_edge_visits = parent.children_and_edge_visits.values()
        parent.visit_count = 1 + sum(
            edge_visits for (_, edge_visits) in children_and_edge_visits
        )
        parent.cumulative_value = (1 / parent.visit_count) * (
            parent.utility_values[parent.game_state.player]
            + sum(
                child.cumulative_value * edge_visits
                for (child, edge_visits) in children_and_edge_visits
            )
        )

policy = {
    action: edge_visits / (current_node.visit_count - 1)
    for action, (
        _,
        edge_visits,
    ) in current_node.children_and_edge_visits.items()
}
policy


{<simulator.game.connect.Action at 0x2cdbac270>: 0.9494949494949495,
 <simulator.game.connect.Action at 0x2cdbad190>: 0.020202020202020204,
 <simulator.game.connect.Action at 0x2cdbaf8f0>: 0.020202020202020204,
 <simulator.game.connect.Action at 0x2cdbafa70>: 0.010101010101010102}

In [50]:
current_node.game_state.grid

array([[ 0,  0,  0, -1, -1,  1, -1],
       [-1, -1, -1,  1, -1,  1, -1],
       [-1, -1, -1,  1, -1,  1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1]], dtype=int8)

In [51]:
[policy], [value] = model.predict(
    [current_node.game_state.action_at(3).sample_next_state()]
)
policy, value, current_node.game_state.reward


({}, [-0.7607280015945435, 0.7607279419898987], array([0., 0.], dtype=float32))

In [62]:
from simulator.game.connect import Action, Config, State

from alphazero_implementation.models.games.connect4 import CNNModel

config = Config(6, 7, 4)


path = "/Users/pveron/Code/alphazero-implementation/checkpoints/run_156/model-epoch=1999.ckpt"
path = "/Users/pveron/Code/alphazero-implementation/lightning_logs/alphazero/run_168_iter200_episodes100_sims100/checkpoints/epoch=1999-step=1431360.ckpt"

model = CNNModel.load_from_checkpoint(  # type: ignore[arg-type]
    path,
    height=config.height,
    width=config.width,
    max_actions=config.width,
    num_players=config.num_players,
)
model.eval()


CNNModel(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (shared_layers): Sequential(
    (0): Linear(in_features=10752, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (policy_head): Linear(in_features=512, out_features=7, bias=True)
  (value_head): Linear(in_features=512, out_features=2, bias=True)
)

In [63]:
state = config.sample_initial_state()
json = {
    "config": {"count": 4, "height": 6, "width": 7},
    "grid": [
        [0, 0, 0, -1, -1, 1, -1],
        [-1, -1, -1, -1, -1, 1, -1],
        [-1, -1, -1, -1, -1, 1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
    ],
    "player": 0,
}

state = State.from_json(json)

In [64]:
def raw_policy(state: State) -> dict[Action, float]:
    [policy], _ = model.predict([state])
    return policy


In [65]:
from alphazero_implementation.mcts.agent import MCTSAgent


def improved_policy_v1(state: State) -> dict[Action, float]:
    mcts = MCTSAgent(model, 1, 100, state)

    episodes = mcts.run()
    policy = episodes[0].samples[-1].policy
    return policy


In [66]:
from alphazero_implementation.mcts_v2.mcts import AlphaZeroMCTS
from alphazero_implementation.mcts_v2.node import Node


def improved_policy_v2(state: State) -> dict[Action, float]:
    mcts = AlphaZeroMCTS(model)

    root = Node(state)
    policy = mcts.run(root, 100)
    return policy

In [67]:
print("State:")
print(state.grid)
print()
# Compare the three different policy functions
state_policy = raw_policy(state)
print("Raw policy from model:")
print(state_policy)
print()

mcts_policy_v1 = improved_policy_v1(state)
print("Improved policy from MCTS v1:")
print(mcts_policy_v1)
print()

mcts_policy_v2 = improved_policy_v2(state)
print("Improved policy from MCTS v2:")
print(mcts_policy_v2)


State:
[[ 0  0  0 -1 -1  1 -1]
 [-1 -1 -1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]]

Raw policy from model:
{<simulator.game.connect.Action object at 0x2cdbad450>: 0.002391638932749629, <simulator.game.connect.Action object at 0x2cdbafc30>: 0.00013282954751048237, <simulator.game.connect.Action object at 0x2cdbae430>: 0.0016656157094985247, <simulator.game.connect.Action object at 0x2cdbad570>: 0.6138741374015808, <simulator.game.connect.Action object at 0x2cdbaca10>: 0.0007002202328294516, <simulator.game.connect.Action object at 0x2cdbae1d0>: 0.3806518316268921, <simulator.game.connect.Action object at 0x2cdbaddb0>: 0.0005836421623826027}

run took 0.23 seconds to run.
Improved policy from MCTS v1:
{<simulator.game.connect.Action object at 0x2cdb8dc90>: 0.0056179775280898875, <simulator.game.connect.Action object at 0x2cdb8ccd0>: 0.9943820224719101}

Improved policy from MCTS v2:
{<simulator.game.connect.Action objec

In [68]:
def print_policy(policy: dict[Action, float]):
    for action, prob in policy.items():
        print(f"{action.column}: {prob:.4f}")


print("Raw policy:")
print_policy(state_policy)
print()

print("MCTS v1 policy:")
print_policy(mcts_policy_v1)
print()

print("MCTS v2 policy:")
print_policy(mcts_policy_v2)


Raw policy:
0: 0.0024
1: 0.0001
2: 0.0017
3: 0.6139
4: 0.0007
5: 0.3807
6: 0.0006

MCTS v1 policy:
0: 0.0056
5: 0.9944

MCTS v2 policy:
0: 0.0000
1: 0.0000
2: 0.0000
3: 0.9600
4: 0.0000
5: 0.0300
6: 0.0000
